Naive Bayes est un algorithme d'apprentissage automatique populaire utilisé pour les tâches de classification, en particulier dans le contexte de la classification de texte, comme la détection de spam. Il est basé sur le théorème de Bayes avec une hypothèse d'indépendance entre les caractéristiques. Malgré sa simplicité, Naive Bayes donne souvent de bons résultats et est efficace en termes de calcul.

Voici un bref aperçu de la façon dont fonctionne Naive Bayes pour la classification de spam :

1. **Compréhension du théorème de Bayes** :

   Le théorème de Bayes calcule la probabilité d'un événement en fonction des probabilités d'autres événements. Dans le contexte de la classification, il est utilisé pour estimer la probabilité d'une certaine classe (dans ce cas, "spam" ou "non spam") étant donné les caractéristiques (mots ou attributs) d'une donnée.

   La formule est :

   $$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

   - $P(A|B)$ : Probabilité de la classe A sachant l'évidence B.
   - $P(B|A)$ : Probabilité de l'évidence B sachant la classe A.
   - $P(A)$ : Probabilité de la classe A.
   - $P(B)$ : Probabilité de l'évidence B.

2. **L'hypothèse naïve** :

   La partie "naïve" de Naive Bayes vient de l'hypothèse d'indépendance des caractéristiques. Il suppose que la présence ou l'absence d'une caractéristique particulière est indépendante de la présence ou de l'absence de toute autre caractéristique. Cela n'est souvent pas vrai dans les données du monde réel, mais l'algorithme peut tout de même être étonnamment efficace.

3. **Représentation du texte** :

   Dans le contexte de la classification de spam, les données textuelles doivent être converties en un format numérique que l'algorithme peut comprendre. Cela se fait souvent à l'aide de techniques comme TF-IDF (Fréquence du terme-Fréquence inverse du document) ou la vectorisation de comptage.

4. **Entraînement** :

   Pendant la phase d'entraînement, l'algorithme apprend les probabilités nécessaires à la classification. Il calcule les probabilités de chaque classe (spam et non spam) ainsi que les probabilités de chaque caractéristique étant donné chaque classe.

5. **Prédiction** :

   Lors de la classification de nouvelles données (par exemple, un e-mail), l'algorithme calcule la probabilité qu'elles appartiennent à chaque classe en fonction des caractéristiques. Il sélectionne ensuite la classe avec la probabilité la plus élevée comme classe prédite.

   $$P(\text{Classe} = \text{"spam"} | \text{Caractéristiques}) \propto P(\text{"spam"}) \times \prod_{i=1}^n P(\text{Caractéristique}_i | \text{"spam"})$$

   $$P(\text{Classe} = \text{"non spam"} | \text{Caractéristiques}) \propto P(\text{"non spam"}) \times \prod_{i=1}^n P(\text{Caractéristique}_i | \text{"non spam"})$$

6. **Lissage** :

   Parfois, certains mots ou caractéristiques peuvent ne pas être présents dans les données d'entraînement pour une classe particulière. Cela peut conduire à des probabilités nulles, ce qui causerait des problèmes lors de la classification. Le lissage de Laplace ou d'autres techniques de lissage peuvent être utilisés pour gérer cela.

7. **Évaluation** :

   La performance du modèle est évaluée à l'aide de métriques telles que l'exactitude, la précision, le rappel et le score F1 sur un ensemble de validation ou de test distinct.

N'oubliez pas, Naive Bayes est un algorithme relativement simple et ne surpasse pas toujours des modèles plus complexes. Cependant, il s'entraîne rapidement et peut servir de bon point de départ pour les tâches de classification de texte comme la détection de spam.

<a name='0'></a>
## Importing Functions and Data

In [1]:
import re
#import pdb
from nltk.corpus import stopwords, twitter_samples
from nltk.stem.porter import  PorterStemmer
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd
#import w2_unittest

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\nmadali\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nmadali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# obtenez les ensembles de tweets positifs et négatifs
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# obtenez les ensembles de tweets positifs et négatifs
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# évitez les hypothèses sur la longueur de all_positive_tweets
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))


## 1 - Traiter les données

Pour tout projet d'apprentissage automatique, une fois que vous avez collecté les données, la première étape consiste à les traiter pour apporter des entrées utiles à votre modèle.
- **Supprimer le bruit** : vous souhaiterez d'abord supprimer le bruit de vos données, c'est-à-dire supprimer les mots qui ne vous disent pas grand-chose sur le contenu. Ceux-ci incluent tous les mots courants comme « Je, vous, êtes, est, etc... » qui ne nous donneraient pas suffisamment d'informations sur le sentiment.

- Nous supprimerons également les tickers boursiers, les symboles de retweet, les hyperliens et les hashtags, car ils ne peuvent pas vous donner beaucoup d'informations sur le sentiment.

- Vous souhaitez également supprimer toute la ponctuation d'un tweet. La raison en est que nous voulons traiter les mots avec ou sans ponctuation comme le même mot, au lieu de traiter « heureux », « heureux ? », « heureux ! », « heureux » et « heureux ». comme des mots différents.

- Enfin, vous souhaitez utiliser la recherche radicale pour ne suivre qu'une seule variation de chaque mot. En d’autres termes, nous traiterons « motivation », « motivé » et « motiver » de la même manière en les regroupant au sein du même radical « motiv- ».

Nous vous avons donné la fonction `process_tweet` qui fait cela pour vous.

In [3]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False,        strip_handles=True,reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  
                word not in string.punctuation): 
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [4]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# print cleaned tweet
print(process_tweet(custom_tweet))

['hello', 'great', 'day', ':)', 'good', 'morn']



### 1.1 - Implémentation de vos fonctions d'assistance

Pour vous aider à entraîner votre modèle bayésien naïf, vous devrez calculer un dictionnaire où les clés sont un tuple (mot, étiquette) et les valeurs sont la fréquence correspondante. Notez que les étiquettes que nous utiliserons ici sont 1 pour positif et 0 pour négatif.

Vous implémenterez également une fonction d'aide à la recherche qui prend dans le dictionnaire « freqs », un mot et une étiquette (1 ou 0) et renvoie le nombre de fois que ce mot et ce tuple d'étiquette apparaissent dans la collection de tweets.

Par exemple : étant donné une liste de tweets `["je suis plutôt excité", "vous êtes plutôt heureux"]` et le label 1, la fonction renverra un dictionnaire contenant les paires clé-valeur suivantes :

{
     ("plutôt", 1): 2,
     ("heureux", 1) : 1,
     ("exciter", 1) : 1
}

- Remarquez comment pour chaque mot de la chaîne donnée, la même étiquette 1 est attribuée à chaque mot.
- Notez que les mots "je" et "suis" ne sont pas enregistrés, car ils ont été supprimés par process_tweet car il s'agit d'un mot vide.
- Remarquez comment le mot "plutôt" apparaît deux fois dans la liste des tweets, et sa valeur de comptage est donc 2.




In [5]:


def count_tweets(result, tweets, ys):
    '''
     Saisir:
         résultat : un dictionnaire qui servira à mapper chaque paire à sa fréquence
         tweets : une liste de tweets
         ys : une liste correspondant au sentiment de chaque tweet (soit 0, soit 1)
     Sortir:
         résultat : un dictionnaire mappant chaque paire à sa fréquence
     '''
    
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # définit la clé, qui est le tuple de mots et d'étiquettes
            pair = (word, y)
            
            # si la clé existe dans le dictionnaire, incrémente le décompte
            if pair in result:
                result[pair] += 1

            # sinon, si la clé est nouvelle, ajoutez-la au dictionnaire et définissez le nombre sur 1
            else:
                result[pair] = 1


    return result

In [6]:
# Tester votre fonction

result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}


## 2 - Entraînez votre modèle à l'aide de Naive Bayes

Naive Bayes est un algorithme qui pourrait être utilisé pour l'analyse des sentiments. L'entraînement prend peu de temps et le temps de prédiction est également court.

#### Alors, comment former un classificateur Naive Bayes ?
- La première partie de la formation d'un classificateur bayésien naïf consiste à identifier le nombre de classes dont vous disposez.
- Vous créerez une probabilité pour chaque classe.
$P(D_{pos})$ est la probabilité que le document soit positif.
$P(D_{neg})$ est la probabilité que le document soit négatif.
Utilisez les formules comme suit et stockez les valeurs dans un dictionnaire :

$$P(D_{pos}) = \frac{D_{pos}}{D}\tag{1}$$

$$P(D_{nég}) = \frac{D_{nég}}{D}\tag{2}$$

Où $D$ est le nombre total de documents, ou de tweets dans ce cas, $D_{pos}$ est le nombre total de tweets positifs et $D_{neg}$ est le nombre total de tweets négatifs.

#### Prior et Logprior

La probabilité a priori représente la probabilité sous-jacente dans la population cible qu'un tweet soit positif ou négatif. En d’autres termes, si nous n’avions aucune information spécifique et si nous choisissions aveuglément un tweet parmi la population, quelle est la probabilité qu’il soit positif ou négatif ? C'est le "prior".

Le prior est le rapport des probabilités $\frac{P(D_{pos})}{P(D_{neg})}$.
Nous pouvons prendre le journal du prior pour le redimensionner, et nous appellerons cela le logprior

$$\text{logprior} = journal \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = journal \left( \frac{D_{pos}}{D_ {nég}} \right)$$.

Notez que $log(\frac{A}{B})$ est identique à $log(A) - log(B)$. Ainsi, le logprior peut également être calculé comme la différence entre deux journaux :

$$\text{logprior} = \log (P(D_{pos})) - \log (P(D_{neg})) = \log (D_{pos}) - \log (D_{neg})\ balise{3}$$

#### Probabilité positive et négative d'un mot
Pour calculer la probabilité positive et la probabilité négative d'un mot spécifique du vocabulaire, nous utiliserons les entrées suivantes :

- $freq_{pos}$ et $freq_{neg}$ sont les fréquences de ce mot spécifique dans la classe positive ou négative. En d’autres termes, la fréquence positive d’un mot est le nombre de fois où le mot est compté avec l’étiquette 1.
- $N_{pos}$ et $N_{neg}$ sont respectivement le nombre total de mots positifs et négatifs pour tous les documents (pour tous les tweets).
- $V$ est le nombre de mots uniques dans l'ensemble des documents, pour toutes les classes, qu'elles soient positives ou négatives.

Nous les utiliserons pour calculer la probabilité positive et négative d'un mot spécifique en utilisant cette formule :

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{nég}) = \frac{freq_{nég} + 1}{N_{nég} + V}\tag{5} $$

Notez que nous ajoutons le "+1" au numérateur pour le lissage additif. Cet [article wiki](https://en.wikipedia.org/wiki/Additive_smoothing) en explique plus sur le lissage additif.

#### Enregistrer la probabilité
Pour calculer la log-vraisemblance de ce même mot, nous pouvons implémenter les équations suivantes :

$$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$$

##### Créer un dictionnaire `freqs`
- Étant donné votre fonction `count_tweets`, vous pouvez calculer un dictionnaire appelé `freqs` qui contient toutes les fréquences.
- Dans ce dictionnaire `freqs`, la clé est le tuple (mot, label)
- La valeur est le nombre de fois qu'elle est apparue.

Nous utiliserons ce dictionnaire dans plusieurs parties de ce devoir.

In [7]:
# Construire le dictionnaire de fréquences pour des utilisations ultérieures
freqs = count_tweets({}, train_x, train_y)

In [8]:


def train_naive_bayes(freqs, train_x, train_y):
    '''
     Saisir:
         freqs : dictionnaire de (mot, étiquette) à la fréquence d'apparition du mot
         train_x : une liste de tweets
         train_y : une liste de labels correspondant aux tweets (0,1)
     Sortir:
         logprior : le journal antérieur. (équation 3 ci-dessus)
         logvraisemblance : le log de vraisemblance de votre équation naïve de Bayes. (équation 6 ci-dessus)
     '''
    loglikelihood = {}
    logprior = 0

    

    # calculer V, le nombre de mots uniques dans le vocabulaire
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculer N_pos, N_neg, V_pos, V_neg
    N_pos = N_neg = 0    
    for pair in freqs.keys():
        # si le label est positif (supérieur à zéro)
        if pair[1] > 0:

            # Augmentez le nombre de mots positifs du nombre pour cette paire (mot, étiquette)
            N_pos += freqs[pair]

        # sinon, le label est négatif
        else:

            # incrémente le nombre de mots négatifs du nombre pour cette paire (mot, étiquette)
            N_neg += freqs[pair]
    
    # Calculez D, le nombre de documents
    D = train_y.shape[0]

    # Calculez D, le nombre de documents
    D_pos = train_y[train_y == 1].shape[0]

    # Calculez D_neg, le nombre de documents négatifs
    D_neg = train_y[train_y == 0].shape[0]

    # Calculer le logprior
    logprior = np.log(D_pos/D_neg)
    
    # Pour chaque mot du vocabulaire...
    for word in vocab:
        # obtenir la fréquence positive et négative du mot
        freq_pos = freqs.get((word, 1), 0)
        freq_neg = freqs.get((word, 0), 0)

        # calculer la probabilité que chaque mot soit positif et négatif
        p_w_pos = (freq_pos+1)/(N_pos+V)
        p_w_neg = (freq_neg+1)/(N_neg+V)

        # calculer la vraisemblance du mot
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)

   
    return logprior, loglikelihood

In [9]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
9086


<a name='3'></a>
## 3 – Testez votre Naive Bayes

Maintenant que nous avons le « logprior » et le « loglikelihood », nous pouvons tester la fonction naïve de Bayes en faisant des prédictions sur certains tweets !

<a name='ex-3'></a>
### Exercice 3 - naive_bayes_predict
Implémentez `naive_bayes_predict`.

**Instructions**:
Implémentez la fonction `naive_bayes_predict` pour faire des prédictions sur les tweets.
* La fonction prend en compte le « tweet », le « logprior », le « loglikelihood ».
* Il renvoie la probabilité que le tweet appartienne à la classe positive ou négative.
* Pour chaque tweet, résumez les log-vraisemblances de chaque mot du tweet.
* Ajoutez également le logprior à cette somme pour obtenir le sentiment prévu de ce tweet.

$$ p = logprior + \sum_i^N (loglikelihood_i)$$

#### Note
Notez que nous calculons le prior à partir des données de formation et que les données de formation sont réparties également entre les étiquettes positives et négatives (4 000 tweets positifs et 4 000 négatifs). Cela signifie que le rapport du positif au négatif est de 1 et que le logprior est de 0.

La valeur de 0,0 signifie que lorsque nous ajoutons le logprior au log de vraisemblance, nous ajoutons simplement zéro au log de vraisemblance. Cependant, n'oubliez pas d'inclure le logprior, car chaque fois que les données ne sont pas parfaitement équilibrées, le logprior sera une valeur non nulle.

In [10]:


def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
     Saisir:
         tweet : une chaîne
         logprior : un nombre
         logvraisemblance : un dictionnaire de mots correspondant à des nombres
     Sortir:
         p : la somme de toutes les logliklihoods de chaque mot du tweet (s'il est trouvé dans le dictionnaire) + logprior (un nombre)

     '''
    
    # traiter le tweet pour obtenir une liste de mots
    word_l = process_tweet(tweet)

    # initialiser la probabilité à zéro
    p = 0

    # ajouter le logprior
    p += logprior

    for word in word_l:

        # vérifie si le mot existe dans le dictionnaire log-vraisemblance
        if word in loglikelihood:
            # ajoute la probabilité logarithmique de ce mot à la probabilité
            p += loglikelihood[word]

    
    return p

In [11]:
my_tweet = 'She smiled.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1.5736965101941158


In [13]:
# Experiment with your own tweet.
my_tweet = 'He laughed.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is -0.14907008754698764


<a name='ex-4'></a>
### Exercice 4 - test_naive_bayes
Implémentez test_naive_bayes.

**Instructions**:
* Implémentez `test_naive_bayes` pour vérifier l'exactitude de vos prédictions.
* La fonction prend en compte vos `test_x`, `test_y`, log_prior et loglikelihood
* Il renvoie la précision de votre modèle.
* Tout d'abord, utilisez la fonction `naive_bayes_predict` pour faire des prédictions pour chaque tweet dans text_x.

In [19]:

def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=naive_bayes_predict):
    """
         Saisir:
             test_x : Une liste de tweets
             test_y : les labels correspondants pour la liste des tweets
             logprior : le logprior
             logvraisemblance : un dictionnaire avec les logvraisemblances pour chaque mot
         Sortir:
             précision : (nombre de tweets classés correctement)/(nombre total de tweets)
    """
    accuracy = 0  # return this properly

    
    y_hats = []
    for tweet in test_x:
        # si la prédiction est > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # la classe prédite est 1
            y_hat_i = 1
        else:
            # sinon la classe prédite est 0
            y_hat_i = 0

        # ajoute la classe prédite à la liste y_hats
        y_hats.append(y_hat_i)

    # l'erreur est la moyenne des valeurs absolues des différences entre y_hats et test_y
    error = sum(abs(test_y-y_hats))/len(test_y)

    # La précision est de 1 moins l'erreur
    accuracy = 1-error

   

    return accuracy

In [20]:
print("Précision naïve de Bayes = %0.4f" % (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Précision naïve de Bayes = 0.9940


In [21]:


for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    p
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
#     print(f'{tweet} -> {p:.2f} ({p_category})')
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.15
I am bad -> -1.29
this movie should have been great. -> 2.14
great -> 2.14
great great -> 4.28
great great great -> 6.41
great great great great -> 8.55


In [22]:
# N'hésitez pas à vérifier le sentiment de votre propre tweet ci-dessous
my_tweet = 'you are bad :('
naive_bayes_predict(my_tweet, logprior, loglikelihood)

-8.802285344803796

<a name='4'></a>
## 4 - Filtrer les mots par rapport entre les nombres positifs et négatifs

- Certains mots ont des comptes plus positifs que d'autres et peuvent être considérés comme « plus positifs ». De même, certains mots peuvent être considérés comme plus négatifs que d’autres.
- Une façon pour nous de définir le niveau de positivité ou de négativité, sans calculer le log de vraisemblance, est de comparer la fréquence positive à la fréquence négative du mot.
     - Notez que nous pouvons également utiliser les calculs du log de vraisemblance pour comparer la positivité ou la négativité relative des mots.
- Nous pouvons calculer le rapport des fréquences positives aux fréquences négatives d'un mot.
- Une fois que nous sommes capables de calculer ces ratios, nous pouvons également filtrer un sous-ensemble de mots qui ont un ratio minimum de positivité/négativité ou supérieur.
- De même, on peut également filtrer un sous-ensemble de mots qui ont un rapport positivité/négativité maximum ou inférieur (mots au moins aussi négatifs, voire plus négatifs qu'un seuil donné).

<a name='ex-5'></a>
### Exercice 5 - get_ratio
Implémentez get_ratio.

- Étant donné le dictionnaire de fréquences de mots et d'un mot particulier, utilisez `lookup(freqs,word,1)` pour obtenir le nombre positif du mot.
- De même, utilisez la fonction « recherche » pour obtenir le nombre négatif de ce mot.
- Calculer le rapport des comptes positifs divisés par les comptes négatifs

$$ ratio = \frac{\text{pos_words} + 1}{\text{neg_words} + 1} $$

Où pos_words et neg_words correspondent à la fréquence des mots dans leurs classes respectives.
<table>
    <tr>
        <td>
            <b>Words</b>
        </td>
        <td>
        Positive word count
        </td>
         <td>
        Negative Word Count
        </td>
  </tr>
    <tr>
        <td>
        glad
        </td>
         <td>
        41
        </td>
    <td>
        2
        </td>
  </tr>
    <tr>
        <td>
        arriv
        </td>
         <td>
        57
        </td>
    <td>
        4
        </td>
  </tr>
    <tr>
        <td>
        :(
        </td>
         <td>
        1
        </td>
    <td>
        3663
        </td>
  </tr>
    <tr>
        <td>
        :-(
        </td>
         <td>
        0
        </td>
    <td>
        378
        </td>
  </tr>
</table>

In [23]:


def lookup(freqs, word, label):
    '''
     Saisir:
         freqs : un dictionnaire avec la fréquence de chaque paire (ou tuple)
         mot : le mot à rechercher
         label : le label correspondant au mot
     Sortir:
         n : le nombre de fois que le mot avec son étiquette correspondante apparaît.
     '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n


In [24]:


def get_ratio(freqs, word):
    '''
     Saisir:
         freqs : dictionnaire contenant les mots

     Sortie : un dictionnaire avec les clés « positif », « négatif » et « ratio ».
         Exemple : {'positif' : 10, 'négatif' : 20, 'rapport' : 0,5}
     '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}

    # utilisez lookup() pour trouver les nombres positifs pour le mot (indiqué par l'entier 1)
    pos_neg_ratio['positive'] = lookup(freqs, word, 1)
    
    # utilisez lookup() pour trouver les nombres négatifs pour le mot (indiqué par l'entier 0)
    pos_neg_ratio['negative'] = lookup(freqs, word, 0)
    
    # calculer le rapport entre les nombres positifs et négatifs pour le mot
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive']+1) / (pos_neg_ratio['negative']+1)

    return pos_neg_ratio


In [25]:
get_ratio(freqs, 'happi')

{'positive': 161, 'negative': 18, 'ratio': 8.526315789473685}

<a name='ex-6'></a>
### Exercice 6 - get_words_by_threshold
Implémenter get_words_by_threshold(freqs,label,threshold)

* Si nous définissons l'étiquette sur 1, alors nous rechercherons tous les mots dont le seuil positif/négatif est au moins aussi élevé que ce seuil, ou supérieur.
* Si nous définissons l'étiquette sur 0, alors nous rechercherons tous les mots dont le seuil positif/négatif est au plus aussi bas que le seuil donné, ou inférieur.
* Utilisez la fonction `get_ratio` pour obtenir un dictionnaire contenant le nombre positif, le nombre négatif et le rapport des comptes positifs aux négatifs.
* Ajoutez le dictionnaire `get_ratio` dans un autre dictionnaire, où la clé est le mot et la valeur est le dictionnaire `pos_neg_ratio` qui est renvoyé par la fonction `get_ratio`.
Un exemple de paire clé-valeur aurait cette structure :
```
{'heureux' :
     {'positif' : 10, 'négatif' : 20, 'rapport' : 0,524}
}
```

In [26]:

def get_words_by_threshold(freqs, label, threshold, get_ratio=get_ratio):
    '''
     Saisir:
         fréquences : dictionnaire de mots
         étiquette : 1 pour positif, 0 pour négatif
         seuil : rapport qui sera utilisé comme seuil pour inclure un mot dans le dictionnaire renvoyé
     Sortir:
         word_list : dictionnaire contenant le mot et des informations sur son nombre positif, son nombre négatif et le rapport entre ses nombres positifs et négatifs.
         exemple de paire clé-valeur :
         {'heureux' :
             {'positif' : 10, 'négatif' : 20, 'rapport' : 0,5}
         }
     '''
    word_list = {}

    for key in freqs.keys():
        word, _ = key

        # obtenir le rapport positif/négatif pour un mot
        pos_neg_ratio = get_ratio(freqs, word)

        # si le label est 1 et que le ratio est supérieur ou égal au seuil...
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:
        
            # Ajoutez le pos_neg_ratio au dictionnaire
            word_list[word] = pos_neg_ratio

        # Si le label est 0 et que le pos_neg_ratio est inférieur ou égal au seuil...
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
        
            # Ajoutez le pos_neg_ratio au dictionnaire
            word_list[word] = pos_neg_ratio

        # sinon, ne pas inclure ce mot dans la liste (ne rien faire)

   
    return word_list


In [27]:
# trouver des mots négatifs égaux ou inférieurs à un seuil
get_words_by_threshold(freqs, label=0, threshold=0.05)

{':(': {'positive': 1, 'negative': 3663, 'ratio': 0.0005458515283842794},
 ':-(': {'positive': 0, 'negative': 378, 'ratio': 0.002638522427440633},
 'zayniscomingbackonjuli': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '26': {'positive': 0, 'negative': 20, 'ratio': 0.047619047619047616},
 '>:(': {'positive': 0, 'negative': 43, 'ratio': 0.022727272727272728},
 'lost': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '♛': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 '》': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 'beli̇ev': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'wi̇ll': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'justi̇n': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｓｅｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｍｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776}}

In [28]:
# trouver des mots positifs au niveau ou au-dessus d'un seuil
get_words_by_threshold(freqs, label=1, threshold=10)

{'followfriday': {'positive': 23, 'negative': 0, 'ratio': 24.0},
 'commun': {'positive': 27, 'negative': 1, 'ratio': 14.0},
 ':)': {'positive': 2847, 'negative': 2, 'ratio': 949.3333333333334},
 'flipkartfashionfriday': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':d': {'positive': 498, 'negative': 0, 'ratio': 499.0},
 ':p': {'positive': 104, 'negative': 0, 'ratio': 105.0},
 'influenc': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':-)': {'positive': 543, 'negative': 0, 'ratio': 544.0},
 "here'": {'positive': 20, 'negative': 0, 'ratio': 21.0},
 'youth': {'positive': 14, 'negative': 0, 'ratio': 15.0},
 'bam': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'warsaw': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'shout': {'positive': 11, 'negative': 0, 'ratio': 12.0},
 ';)': {'positive': 22, 'negative': 0, 'ratio': 23.0},
 'stat': {'positive': 51, 'negative': 0, 'ratio': 52.0},
 'arriv': {'positive': 57, 'negative': 4, 'ratio': 11.6},
 'via': {'positive': 60, 'negative': 1, 

Notez la différence entre les ratios positifs et négatifs. Les émojis comme :( et les mots comme « moi » ont tendance à avoir une connotation négative. D'autres mots comme heureux, communauté, arrive, ont tendance à être trouvés dans les tweets positifs.

## 5 - Analyse des erreurs

Dans cette partie, vous verrez quelques tweets que votre modèle a mal classés. Pourquoi pensez-vous que les erreurs de classification se sont produites ? Votre modèle bayésien naïf a-t-il formulé des hypothèses ?

In [29]:
# Une analyse d'erreur effectuée pour vous
print('Tweet prédit par la vérité')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            process_tweet(x)).encode('ascii', 'ignore')))

Tweet prédit par la vérité
1	0.00	b''
1	0.00	b'truli later move know queen bee upward bound movingonup'
1	0.00	b'new report talk burn calori cold work harder warm feel better weather :p'
1	0.00	b'harri niall 94 harri born ik stupid wanna chang :d'
1	0.00	b''
1	0.00	b''
1	0.00	b'park get sunlight'
1	0.00	b'uff itna miss karhi thi ap :p'
0	1.00	b'hello info possibl interest jonatha close join beti :( great'
0	1.00	b'u prob fun david'
0	1.00	b'pat jay'
0	1.00	b'whatev stil l young >:-('


<a name='6'></a>
## 6 - Prédisez avec votre propre Tweet

Dans cette partie, vous pouvez prédire le sentiment de votre propre tweet

In [30]:
# Testez avec votre propre tweet - n'hésitez pas à modifier `my_tweet`
my_tweet = 'I am happy because I am learning :)'

p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

9.573774904705935
